<a href="https://colab.research.google.com/github/upadhyayprakash/eva4/blob/master/S12/EVA4_Session_12_TinyImageNet_RESNet18_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Session-12 Assignment**

### **Assignment A**:
Download this TINY IMAGENET (Links to an external site.) dataset. 
Train ResNet18 on this dataset (70/30 split) for 50 Epochs. Target 50%+ Validation Accuracy. 
Submit Results. Of course, you are using your own package for everything. You can look at this (Links to an external site.) for reference. 

Target Validation Accuracy: 50%. 
### **Assignment B**:
- Download 50 images of dogs. 
- Use this (Links to an external site.) to annotate bounding boxes around the - dogs.
- Download JSON file. 
- Describe the contents of this JSON file in FULL details (you don't need to - describe all 10 instances, anyone would work). 
- Refer to this tutorial (Links to an external site.). Find out the best total - numbers of clusters. Upload link to your Colab File uploaded to GitHub. 

### **These are the questions in S12-Assignment-Solution**
- What is your final accuracy?
- Share the Github link to your ResNet-Tiny-ImageNet code. All the logs must be visible. 

- Describe the contents of the JSON file in detail. You need to explain each element in detail. 
- Share the link to your Github file where you have calculated the best K clusters for your 50 dog dataset. 

- Share the link to your 50 Dog Images Folder on GitHub
- Share the link to your JSON file on GitHub

#### **RESULT**
GitHub Source: https://github.com/upadhyayprakash/eva4/tree/master/S12

#### **Summary**

| Metric          | Values         |
| --------------- | -------------- |
| Train/Test Split| **70/30 Split**  |
| Network Arch.   | **RESNET-18**  |
| # of Epochs     | **50**         |
| # of Parameters | **11,173,962** |
| Final Accuracy  | **__.__%**    |
| Optimizer | **SGD** with Momentum |
| Scheduler | **ReduceLROnPlateau** |
| K-MEANS Method | **Elbow** Method |

#### Loading the Libraries

In [0]:
# Setting up the module import directory
# import sys
# sys.path.append('/home/prakash/Prakash/EVA4/Session-12/Notebooks')

### Link the Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
sys.path.append('drive/My Drive/EVA4/Session-12/Notebooks/')
!pip install albumentations
!pip install 

ERROR: You must give at least one requirement to install (see "pip help install")


### Download the TinyImageNet-200 Dataset

In [3]:
import os.path
from os import path
import zipfile
  
import requests
source_url = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
target_path = 'drive/My Drive/EVA4/Session-12/Notebooks/data'

# Check if Dataset already exists or not.
if(not path.exists('drive/My Drive/EVA4/Session-12/Notebooks/data/tiny-imagenet-200.zip')):
  print('Dataset doesn\'t Exist. Downloading...')
  r = requests.get(source_url, stream = True)
  with open(target_path+"/tiny-imagenet-200.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
      if block:  
        file.write(block)
  
  print('Extracting the ZIP')
  # !unzip -uq "drive/My Drive/EVA4/Session-12/Notebooks/data/tiny-imagenet-200.zip" -d "drive/My Drive/EVA4/Session-12/Notebooks/data"
  
  zip_ref = zipfile.ZipFile("drive/My Drive/EVA4/Session-12/Notebooks/data/tiny-imagenet-200.zip", 'r')
  zip_ref.extractall("drive/My Drive/EVA4/Session-12/Notebooks/data")
  zip_ref.close()
  print('Extraction FINISHED!')

else:
  print('Dataset already Exists.')

Dataset already Exists.


### Preparing the Data Directory Structure

In [0]:
from datapreparation.tiny_imagenet_200_prepare import prepare_data

# prepare_data()

In [0]:
# Unzipping the train_set.zip and test_set.zip
# print('Extracting the train_set.zip')
# zip_ref = zipfile.ZipFile("drive/My Drive/EVA4/Session-12/Notebooks/data/tiny-imagenet-200/train_set.zip", 'r')
# zip_ref.extractall("drive/My Drive/EVA4/Session-12/Notebooks/data/tiny-imagenet-200")
# zip_ref.close()
# print('Extraction FINISHED for train_set.zip!')

# print('Extracting the test_set.zip')
# zip_ref = zipfile.ZipFile("drive/My Drive/EVA4/Session-12/Notebooks/data/tiny-imagenet-200/test_set.zip", 'r')
# zip_ref.extractall("drive/My Drive/EVA4/Session-12/Notebooks/data/tiny-imagenet-200")
# zip_ref.close()
# print('Extraction FINISHED for test_set.zip!')

In [0]:
# for inline graph plotting
%matplotlib inline

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import OneCycleLR, ReduceLROnPlateau, StepLR, LambdaLR # LR Scheduler
from torchsummary import summary

#### Checking if GPU(CUDA) is supported

In [6]:
cuda = torch.cuda.is_available() # returns True/False
print('GPU Available?', cuda)

# initiate the Network
device = torch.device("cuda" if cuda else "cpu")

GPU Available? True


#### Loading and Transforming the CIFAR10 Dataset

In [10]:
# MODULES: Train and Test Data Loaders modules
from dataloaders.tiny_imagenet_200_loader import *

trainloader = getTrainLoader()
testloader = getTestLoader()

classes = getClasses()

print('Classes: ')
print(classes)

Classes: 
['n02124075', 'n04067472', 'n04540053', 'n04099969', 'n07749582', 'n01641577', 'n02802426', 'n09246464', 'n07920052', 'n03970156', 'n03891332', 'n02106662', 'n03201208', 'n02279972', 'n02132136', 'n04146614', 'n07873807', 'n02364673', 'n04507155', 'n03854065', 'n03838899', 'n03733131', 'n01443537', 'n07875152', 'n03544143', 'n09428293', 'n03085013', 'n02437312', 'n07614500', 'n03804744', 'n04265275', 'n02963159', 'n02486410', 'n01944390', 'n09256479', 'n02058221', 'n04275548', 'n02321529', 'n02769748', 'n02099712', 'n07695742', 'n02056570', 'n02281406', 'n01774750', 'n02509815', 'n03983396', 'n07753592', 'n04254777', 'n02233338', 'n04008634', 'n02823428', 'n02236044', 'n03393912', 'n07583066', 'n04074963', 'n01629819', 'n09332890', 'n02481823', 'n03902125', 'n03404251', 'n09193705', 'n03637318', 'n04456115', 'n02666196', 'n03796401', 'n02795169', 'n02123045', 'n01855672', 'n01882714', 'n02917067', 'n02988304', 'n04398044', 'n02843684', 'n02423022', 'n02669723', 'n04465501', '

#### Visualize the training data

In [0]:
# import visualize_data

# visualize_data.visualize_batch(trainloader, classes)

### Creating the Network Object

In [11]:
# Loading the Network
# from network import Net
from models.resnet import ResNet18

net = ResNet18().to(device)
summary(net, input_size=(3, 64, 64)) # Network Summary

RuntimeError: ignored

### Define Loss Criterion, Optimizer

In [0]:
# define Loss Function, Optimizer and Step learning rate

loss_fun = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-2)

### Training Program

In [13]:
# Model Trainer
import model_trainer

# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, min_lr=1e-4)

# scheduler = one_cycle_policy_scheduler

EPOCHS = 50
# Training and Testing the model
traintestresult = model_trainer.fit_model(net, EPOCHS, device, trainloader, testloader, classes, loss_fun, optimizer, scheduler)
model = traintestresult['model']

testing module loaded
EPOCH: 1
 [================================================================>]  Step: 161ms | Tot: 2m533ms | Train Loss: 4.956 | Train Acc: 3.471% (2777/80000) 157/157 
testing module CALLED!
step-1


RuntimeError: ignored

### Train/Test Visualization
Visualizing Training and Testing Results

In [0]:
# extract the training/testing metrics
val_acc = traintestresult['val_acc']
val_loss = traintestresult['val_loss']
train_acc = traintestresult['train_acc']
train_loss = traintestresult['train_loss']

# visualize the training/testing stats
visualize_data.visualize_train_test_acc(train_acc, val_acc)

### Class-wise Evaluation Result

In [0]:
import class_wise_results

class_wise_results.show_class_wise_results(model, testloader, device, classes)

### Plotting Mis-classified Sample Objects

In [0]:
import misclassifieds
# Plotting mis-classified test
failed_samples = misclassifieds.plot_mis_classified_objects(model, device, classes, testloader)

### GradCam Visualization
Visualizing the failed test samples using GradCAM and GradCAM++

In [0]:
# Visualize the GradCam 
from grad_cam_vis import visualize_grad_cam
%matplotlib inline

# Visualizing the GradCAM in the format of:
# Original Image, It's heatmap, heatmap of GradCAM++, Final Result, Final Result using GradCAM++
visualize_grad_cam(model, failed_samples, classes)